In [ ]:
import pathlib
import sys
import os
root_path = pathlib.Path().absolute().parent
sys.path.append(str(root_path))

from src.train import run_experiment
from src.models.dqn import SpatialDQN
from src.replay_memory import FastReplayBuffer
from src.env import FourRoomEnvWithTagging
from src.featurizers import PerspectiveFeaturizer, GlobalFeaturizer
from src.visualize import StateSequenceVisualizer

import torch

torch.set_printoptions(precision=3, sci_mode=False, linewidth=200)

In [ ]:
BUF_SIZE = 3000
N_IMPOSTERS = 1
N_JOBS = 5
N_CREW = 4
SEQUENCE_SIZE = 2
env = FourRoomEnvWithTagging(n_imposters=N_IMPOSTERS, n_crew=N_CREW, n_jobs=N_JOBS, debug=False)

In [ ]:
imposter_model_config = {
    "input_image_size": env.n_cols,
    "non_spatial_input_size": 20,
    "n_channels": [7, 5, 3],
    "strides": [1, 1],
    "paddings": [1, 1],
    "kernel_sizes": [3, 3],
    "rnn_layers": 3,
    "rnn_hidden_dim": 64,
    "rnn_dropout": 0.2,
    "mlp_hidden_layer_dims": [16, 16],
    "n_actions": env.n_imposter_actions
}

crew_model_config = {
    "input_image_size": env.n_cols,
    "non_spatial_input_size": 20,
    "n_channels": [7, 5, 3],
    "strides": [1, 1],
    "paddings": [1, 1],
    "kernel_sizes": [3, 3],
    "rnn_layers": 3,
    "rnn_hidden_dim": 64,
    "rnn_dropout": 0.2,
    "mlp_hidden_layer_dims": [16, 16],
    "n_actions": env.n_crew_actions
}

In [ ]:
run_experiment(
    env=env, 
    num_steps=100000,
    imposter_dqn_args=imposter_model_config,
    crew_dqn_args={},
    imposter_dqn_type='spatial_dqn',
    crew_dqn_type='random', 
    featurizer_type='global',
    sequence_length = 10,
    replay_buffer_size=10_000,
    replay_prepopulate_steps=1_000,
    batch_size=4,
    gamma=0.99,
    scheduler_start_eps=1,
    scheduler_end_eps=0.05,
    scheduler_time_steps=100_000,
    train_imposter=True,
    train_crew=False,
    imposter_pretrained_model_path=None,
    crew_pretrained_model_path=None,
    experiment_save_path=root_path / "model_registry" / "tests",
    optimizer_type = "Adam",
    learning_rate=0.001,
    train_step_interval=5,
    num_checkpoint_saves=2,
)

In [ ]:
run_experiment(
    env=env, 
    num_steps=100000,
    imposter_dqn_args={},
    crew_dqn_args=crew_model_config,
    imposter_dqn_type='random',
    crew_dqn_type='spatial_dqn', 
    featurizer_type='global',
    sequence_length = 3,
    replay_buffer_size=3000,
    replay_prepopulate_steps=1_000,
    batch_size=4,
    gamma=0.99,
    scheduler_start_eps=1,
    scheduler_end_eps=0.05,
    scheduler_time_steps=100_000,
    train_imposter=False,
    train_crew=True,
    imposter_pretrained_model_path=None,
    crew_pretrained_model_path=None,
    experiment_save_path=root_path / "model_registry" / "tests",
    optimizer_type = "Adam",
    learning_rate=0.001,
    train_step_interval=5,
    num_checkpoint_saves=2,
)

In [ ]:

# THIS IS A WORKING MODEL SETUP!!!

imposter_model = SpatialDQN(
            input_image_size=env.n_cols,
            non_spatial_input_size=23,
            n_channels=[8, 5, 3],
            strides=[1, 1],
            paddings=[1, 1],
            kernel_sizes=[3, 3],
            rnn_layers=3,
            rnn_hidden_dim=64,
            rnn_dropout=0.2,
            mlp_hidden_layer_dims=[16, 16],
            n_actions=env.n_imposter_actions,
        )